# I. Import And Cleanup

## 1. Import library

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## 2. Download the page with Toronto postal code

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
        
# Get the data form wikipedia.
html = requests.get(url)

if html.status_code == 200:
    print('Data downloaded!')
else:
    print('Error! Please check the url or your network')

Data downloaded!


## 3. Extract the table from the html respond

In [3]:
# Find the table with class "wikitable sortable" and then get the "tbody" section
tp = BeautifulSoup(html.content, 'html.parser').find('table',{'class':'wikitable sortable'}).find('tbody')

## 4. Convert the wikitable into dataframe

In [4]:
# Find all rows
rows = tp.findAll('tr')

toronto_postal = []

# Go through each row, append text of each row to list. We only get column 1-3 because 0 and 4 is empty
for row in rows:
    toronto_postal.append(row.text.split('\n')[1:4])

# Convert the list to dataframe, use the first element of list as columns name
tp_df = pd.DataFrame(toronto_postal, columns = toronto_postal[0])

# Drop the fisrt row because it's a header
tp_df.drop([0],inplace=True)

# Preview the dataframe
print(tp_df.shape)
tp_df.head()

(287, 3)


,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


## 5. Cleanning

### a. Clean up row **Borough** set as Not assigned

In [5]:
# Get list of indexes for which column Borough has value Not assigned
indexNA = tp_df[tp_df['Borough'] == 'Not assigned'].index

# Delete row with Not assigned Borough
tp_df.drop(indexNA , inplace=True)

# Sort by Postcode column (optional)
#tp_df.sort_values(by='Postcode', inplace=True)

# Reset the index
tp_df.reset_index(drop=True, inplace=True)

# Preview the dataframe
print(tp_df.shape)
tp_df.head()

(210, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


**Let's see how many unique postcode in the data frame**

In [6]:
len(tp_df['Postcode'].unique())

103

### b. Join the neighbourhood with same postcode together

In [7]:
tp_join = tp_df.groupby(['Postcode', 'Borough'], as_index=False)['Neighbourhood'].apply(', '.join).reset_index()

# Set the columns name
tp_join.columns=['PostalCode', 'Borough', 'Neighbourhood']

### c. Set value for row with 'Not assigned' 'Neighbourhood' value

In [8]:
tp_join['Neighbourhood'] = np.where(tp_join['Neighbourhood'] == 'Not assigned', tp_join['Borough'], tp_join['Neighbourhood'])

# Preview the dataframe
tp_join.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### d. How many rows in the dataframe?

In [9]:
tp_join.shape[0]

103

**So we have 103 rows**

# II. Get Geographical Coordinates

In [10]:
# Since geocoder doesn't work (now), we will use the csv file instead
#!wget wget -q https://cocl.us/Geospatial_data -O geospatial_data.csv

# Read the downloaded file into dataframe
with open('geospatial_data.csv') as gd:
    gd_df=pd.read_csv(gd)

# Preview the dataframe
gd_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Since both dataframes have the postal code info, we can merge them together**

In [11]:
tp_join = tp_join.merge(gd_df, left_on='PostalCode', right_on='Postal Code')

# Delete the redundant postal code column
tp_join.drop(['Postal Code'], axis=1, inplace=True)

tp_join.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# III. Segmenting and Clustering

**Fist, let's see how many boroughs and neighborhoods we have here**

We use tp_df dataframe since every neighbourhood in it have its own row

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format
      (len(tp_df['Borough'].unique()), tp_df.shape[0]))

The dataframe has 11 boroughs and 210 neighborhoods.


**Get the latitude and longitude of Toronto**

In [13]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude valuesa

geolocator = Nominatim(user_agent='to_explorer')
location = geolocator.geocode('Toronto, TO')
lat = location.latitude
long = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(lat, long))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### 1. Create a map of Toronto with boroughs

**First, install folium as needed and import it**

In [14]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

**Draw a map**

In [15]:
# create map of Toronto
map_to = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map
for la, lo, boro, neigh in zip(tp_join['Latitude'], tp_join['Longitude'], tp_join['Borough'], tp_join['Neighbourhood']):
    label = '{} | {}'.format(boro, neigh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([la, lo], radius=5,popup=label, color='blue',fill=True,
                       fill_color='#3186cc', fill_opacity=0.7,
                       parse_html=False).add_to(map_to)

map_to

**We will work with neighborhood in the borough that have the word "Toronto"**

In [16]:
to_df=tp_join[tp_join['Borough'].str.contains('Toronto')]
to_df.reset_index(drop=True, inplace=True)

print('Number of neighbourhood:', to_df.shape[0])
to_df.head()

Number of neighbourhood: 39


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


**Let's see how many borough have "Toronto" in them**

In [17]:
br_loc = to_df.groupby('Borough').mean()
print(br_loc.shape[0])
br_loc

4


,Latitude,Longitude
Borough,,
Central Toronto,43.701980,-79.398954
Downtown Toronto,43.654597,-79.383972
East Toronto,43.669436,-79.324654
West Toronto,43.652653,-79.449290


**Create a map with centerpoint at "Downtow Toronto" and marker for 39 neighbourhoods**

In [18]:
cto_lat = br_loc.loc['Downtown Toronto']['Latitude']
cto_long = br_loc.loc['Downtown Toronto']['Longitude']

# create map
cto_map = folium.Map(location=[cto_lat, cto_long], zoom_start=12)

# add markers
for la, lo, boro, neigh in zip(to_df['Latitude'], to_df['Longitude'], to_df['Borough'], to_df['Neighbourhood']):
    label = '{} | {}'.format(boro, neigh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([la, lo], radius=5,popup=label, color='blue',fill=True,
                       fill_color='#3186cc', fill_opacity=0.7,
                       parse_html=False).add_to(cto_map)

cto_map

## 2. Let's explore the neighborhood using Foursquare API

**Impoort library**

In [19]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

**Define Foursquare Credentials and Version**

In [20]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version

**I will replicate the step in previous lab here, with some "shortcut"**

First, we get the first Neighbourhood from the list

In [21]:
to_df.loc[0]

PostalCode                M4E
Borough          East Toronto
Neighbourhood     The Beaches
Latitude              43.6764
Longitude             -79.293
Name: 0, dtype: object

Get its coordinates

In [22]:
n_lat = to_df.loc[0, 'Latitude'] # neighbourhood latitude
n_long = to_df.loc[0, 'Longitude'] # neighbourhood longitude
n_name = to_df.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(n_name, n_lat, n_long))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


**Let's get the top 100 venues that are in The Beaches within a radius of 500 meters**

In [23]:
radius = 500
LIMIT = 100

# construct the url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, VERSION, n_lat, n_long, radius, LIMIT)

# Send the GET request
results = requests.get(url).json()

results

{'meta': {'code': 200, 'requestId': '5e54f1f6b4b6841b8ae1475e'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

I will "borrow" the **get_category_type** function from the Foursquare lab

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and struture it into a *pandas* dataframe

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
2,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
3,Grover Pub and Grub,Pub,43.679181,-79.297215
4,Upper Beaches,Neighborhood,43.680563,-79.292869


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


## 3. It's time to expand to the whole 39 neighbourhood

**Let's create a function to repeat the same process to all the neighborhoods**

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**We will use the above function to create a new dataframe that include all the venues near those neighbourhoods**

In [28]:
to_venues = getNearbyVenues(names=to_df['Neighbourhood'],
                                   latitudes=to_df['Latitude'],
                                   longitudes=to_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

**Check the size of the resulting dataframe**

In [29]:
print(to_venues.shape)
to_venues.head()

(1718, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


**Let's check how many venues were return for each neighbourhood**

In [30]:
to_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,79,79,79,79,79,79
"Chinatown, Grange Park, Kensington Market",84,84,84,84,84,84
Christie,18,18,18,18,18,18


**Let's find out how many unique categories can be curated from all the returned venues**

In [31]:
print('There are {} uniques categories.'.format(len(to_venues['Venue Category'].unique())))

There are 236 uniques categories.


## 4. Analyze Each Neighbourhood

In [32]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighbourhood'] = to_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Let's check new dataframe size**

In [33]:
to_onehot.shape

(1718, 237)

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [34]:
to_grouped = to_onehot.groupby('Neighbourhood').mean().reset_index()

print('Size of new dataframe:' ,to_grouped.shape)
to_grouped.head()

Size of new dataframe: (39, 237)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.02,0.0,...,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.01,0.000000
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,...,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.058824
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.0625,0.125,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000


**Let's print each neighborhood along with the top 5 most common venues**

In [35]:
num_top_venues = 5

for hood in to_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = to_grouped[to_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.04
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.04
2  Seafood Restaurant  0.04
3      Farmers Market  0.04
4         Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.05
4             Stadium  0.05


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.06
1     Garden Center  0.06
2              Park  0.06
3        Comic Shop  0.06
4  Recording Studio  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.12
1  Airport Terminal

**Let's put that into a *pandas* dataframe**

First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = to_grouped['Neighbourhood']

for ind in np.arange(to_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Sushi Restaurant,Restaurant,Concert Hall,Burger Joint,Cosmetics Shop
1,Berczy Park,Coffee Shop,Farmers Market,Cocktail Bar,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Cheese Shop,Café,Pharmacy
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Gym,Bakery,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Smoke Shop,Light Rail Station,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Rental Car Location,Harbor / Marina,Boat or Ferry,Plane,Boutique,Bar,Coffee Shop,Sculpture Garden


## 5. Cluster Neighbourhoods

**Import library**

In [38]:
from sklearn.cluster import KMeans

**Run k-means to cluster the neighborhood into 4 clusters**

In [39]:
# set number of clusters
kclusters = 4

to_grouped_clustering = to_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

**Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.**

In [40]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = to_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

to_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Neighborhood,Other Great Outdoors,Health Food Store,Pub,Trail,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Brewery,Bubble Tea Shop,Bakery
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Sandwich Place,Gym,Movie Theater,Brewery,Burrito Place,Pub,Pizza Place,Park,Liquor Store,Sushi Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Construction & Landscaping,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


**Let's visualize the resulting clusters**

In [41]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[cto_lat, cto_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighbourhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' | Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters

**Cluster 1 - The Garden**

In [42]:
to_merged.loc[to_merged['Cluster Labels'] == 0, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,0,Garden,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


**Cluster 2 - The Kitchen**

In [43]:
to_merged.loc[to_merged['Cluster Labels'] == 1, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Neighborhood,Other Great Outdoors,Health Food Store,Pub,Trail,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
1,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Brewery,Bubble Tea Shop,Bakery
2,East Toronto,1,Sandwich Place,Gym,Movie Theater,Brewery,Burrito Place,Pub,Pizza Place,Park,Liquor Store,Sushi Restaurant
3,East Toronto,1,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant
5,Central Toronto,1,Gym,Hotel,Breakfast Spot,Food & Drink Shop,Dog Run,Sandwich Place,Department Store,Park,Costume Shop,Coworking Space
6,Central Toronto,1,Clothing Store,Coffee Shop,Yoga Studio,Fast Food Restaurant,Rental Car Location,Dessert Shop,Bagel Shop,Mexican Restaurant,Salon / Barbershop,Diner
7,Central Toronto,1,Pizza Place,Dessert Shop,Sandwich Place,Gym,Thai Restaurant,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Seafood Restaurant
9,Central Toronto,1,Pub,Light Rail Station,Coffee Shop,Bagel Shop,Health & Beauty Service,Sushi Restaurant,Restaurant,Sports Bar,Fried Chicken Joint,Spa
11,Downtown Toronto,1,Coffee Shop,Park,Restaurant,Italian Restaurant,Bakery,Pizza Place,Chinese Restaurant,Pub,Café,Farmers Market
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Gastropub,Fast Food Restaurant,Pub,Pizza Place,Men's Store


**Cluster 3 - The Sparkling Hot**

In [44]:
to_merged.loc[to_merged['Cluster Labels'] == 2, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,2,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


**Cluster 4 - The Nature**

In [45]:
to_merged.loc[to_merged['Cluster Labels'] == 3, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Park,Construction & Landscaping,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
8,Central Toronto,3,Tennis Court,Park,Playground,Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
10,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
